## data preprossing 

In [1]:

!pip install transformers
!git clone https://github.com/aub-mind/arabert
!pip install -r arabert/requirements.txt
!pip  install naftawayh
!pip install qalsadi
!pip install fuzzywuzzy
!pip install python-Levenshtein
import nltk
#nltk.download('stopwords')
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import ARLSTem
from nltk.stem.isri import ISRIStemmer


import naftawayh
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.1 MB/s eta 0:00:00
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 13.67 MiB/s, done.
Resolving deltas: 100% (339/339), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji

In [2]:
!pip install farasapy==0.0.14
!pip install pyarabic==0.6.14
!pip install emoji==1.6.1
!pip install sentencepiece==0.1.96

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: pyarabic
    Found existing installation: PyArabic 0.6.15
    Uninstalling PyArabic-0.6.15:
      Successfully uninstalled PyArabic-0.6.15
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.0/170.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169312 sha256=c16c680240e788ce777a477a12b988eac41a2c457d5ad6d0373665a0c7d0293c
  Stored in directory: /root/.cache/pip/wheels/3d/c9/af/02caa5725634f27f4e2e43852f67fc9069d014038b236a827e
Successfully built emoji
  Attempting uninstall: emoji
    Found existing install

In [3]:
from qalsadi import lemmatizer 
from transformers import AutoTokenizer, AutoModel

from arabert.preprocess import ArabertPreprocessor
import pandas as pd
from typing import List
from tqdm import tqdm_notebook as tqdm
import torch
import random
import matplotlib.pyplot as plt
import copy
from arabert.preprocess import ArabertPreprocessor
from torch.utils.data import DataLoader, Dataset
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizer, Trainer,
                          TrainingArguments)
from transformers.data.processors.utils import InputFeatures

In [4]:
text= open("/content/1 - Copy (1).txt", encoding='utf-8')
text=text.read()

In [5]:
#Regular Expression
text= re.sub('،', '.',text) # في اللغه العربيه ممكن افصل بين الجمل و بعضها بفصلها لذالك هيتم اسبدالها ب نقصه 




In [6]:
import nltk
nltk.download('punkt')
out="أي . فقد . لأن . لأني . لأنني . بسبب . راجع إلى . لكي . لأجل ذلك . ويعزى إلى . بناءً على . علماً بأن . علماً أني . علمأ أنني . علماً بأنني . إلا . إلا أن . إلا أنني . على الرغم , كما .  بالإضافة إلى . أضف الى ذلك . وقس على ذلك. اذاً . ماعدا . بإستثناء "
out =sent_tokenize(out)
out=[re.sub(r'[^ء-ي ]','',i) for  i in out ]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
#sentences  tokenize
nltk.download('punkt')
text_sentences_= sent_tokenize(text)
#text_sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


***last updata***

In [8]:
for i in range(len(text_sentences_)-1):
  for x in out:
    if text_sentences_[i+1].startswith(x):
      text_sentences_[i]=text_sentences_[i]+text_sentences_[i+1]
      text_sentences_[i+1]=""
text_sentences=[]
for i in text_sentences_:
  if i!= "":
    text_sentences.append(i)
text_sentences


['﴿ (3) المقامة الثالثة وتعريف بالعقيقة  ﴾ \n۞۞۞۞۞۞۞\nكتاب مجمع البحرين لليازجي\n \n حكى سهيل بن عباد قال: بكرت يوماً بكور الزاجر.',
 'في معمعان ناجر.',
 'خوفاً من اصطكاك الهواجر.',
 'فأمعنت في السياحة.',
 'وجعلت أقطع ساحة بعد ساحة.',
 'حتى إذا تخللت بعض الغيطان.',
 'وقد سال عليها مخاط الشيطان.',
 'رأيت كتيبة من الرجال.',
 'على كثيب من الرمال.',
 'فبذلت في شاكلة الجواد المهماز.',
 'ورددت صدور الأرض على الأعجاز.',
 'حتى أدركت القوم.',
 'في منتصف اليوم.',
 'وإذا جنازة قد أودعوها التراب.',
 'وشيخ على دكة قد افتتح الخطاب.',
 'فقال: يا كرام المعاشر والعشائر.',
 'يا كرام المعاشر والعشائر.',
 'وأولي الأبصار والبصائر.',
 'أرأيتم ما أحرج هذا البيت.',
 'وأسمج هذا الميت؟ طالما جد وكد.',
 'واشتد واعتد.',
 'وركب الأهوال.',
 'واحتشد الأموال.',
 'فانظروا أين ما جمع.',
 'وهل أتى بشيء منه إلى هذا المضجع.',
 'وطالما شمخ.',
 'وبذخ.',
 'وأسرف.',
 'واستطرف.',
 'وتأنق في الطعام والشراب.',
 'واستكرم المهاد والثياب.',
 'وتضمخ بالعبير والملاب.',
 'فاعتبروا كيف صار جيفة لا تطاق.',
 'وكريهة لا تستطيع أن تلحظها ا

In [9]:
text_sentences[20:30]

['واشتد واعتد.',
 'وركب الأهوال.',
 'واحتشد الأموال.',
 'فانظروا أين ما جمع.',
 'وهل أتى بشيء منه إلى هذا المضجع.',
 'وطالما شمخ.',
 'وبذخ.',
 'وأسرف.',
 'واستطرف.',
 'وتأنق في الطعام والشراب.']

In [10]:
text_after_pre=" ".join(text_sentences)

In [11]:
def clean_the_sentence(sentences_list):
    new_sentences=[]
    for i in sentences_list:
        value=  re.sub(r'[^ء-ي ]',' ',i) # هنشيل الحشو هناخد الجمله بس من غير اي زياده 
        new_sentences.append(value)
    return new_sentences
    

In [12]:
cleaning_sentences=clean_the_sentence(text_sentences)


In [13]:

print("cleaning sentence after appling  Regular Expression :",cleaning_sentences[0])
print("sentence before appling  Regular Expression :" , text_sentences[0])


cleaning sentence after appling  Regular Expression :       المقامة الثالثة وتعريف بالعقيقة             كتاب مجمع البحرين لليازجي    حكى سهيل بن عباد قال  بكرت يوما  بكور الزاجر 
sentence before appling  Regular Expression : ﴿ (3) المقامة الثالثة وتعريف بالعقيقة  ﴾ 
۞۞۞۞۞۞۞
كتاب مجمع البحرين لليازجي
 
 حكى سهيل بن عباد قال: بكرت يوماً بكور الزاجر.


In [14]:
def get_word_tokenize(sentences):
    list1=[]
    for i in sentences:
        list1.append(word_tokenize(i))
    return list1

In [15]:

sentences_words=get_word_tokenize(cleaning_sentences)
sentences_words[6:10]

[['وقد', 'سال', 'عليها', 'مخاط', 'الشيطان'],
 ['رأيت', 'كتيبة', 'من', 'الرجال'],
 ['على', 'كثيب', 'من', 'الرمال'],
 ['فبذلت', 'في', 'شاكلة', 'الجواد', 'المهماز']]

##  stemming and lemmatization

In [16]:
def steming (lemmatize):
    li=[]
    bag_li=[]
    for i  in sentences_words:
        for x in i:
            st = ISRIStemmer()
            li.append(st.stem(x))
        bag_li.append(li)
        li=[]
    return bag_li
            

In [17]:
def lemmatizing (text_sentences):
    li=[]
    bag_li=[]
    for i  in text_sentences:
        for x in word_tokenize(i) :
            lemmer = lemmatizer.Lemmatizer()
            li.append(lemmer.lemmatize(x))
        bag_li.append(" ".join(li))
        li=[]
    return bag_li

In [18]:
steming_list= steming(sentences_words) 

print("before appling steming: " , sentences_words[6])
print("after appling the sentence: ", steming_list[6])


before appling steming:  ['وقد', 'سال', 'عليها', 'مخاط', 'الشيطان']
after appling the sentence:  ['وقد', 'سال', 'عليها', 'خاط', 'شيط']


In [19]:
lemmatizing_list= lemmatizing(text_sentences[:3]) 

print("before appling lemmatizing: " , text_sentences[11:13])
print("after appling the lemmatizing: ", lemmatizing_list[:])

before appling lemmatizing:  ['حتى أدركت القوم.', 'في منتصف اليوم.']
after appling the lemmatizing:  ['﴿ ( 3 ) مقام ثالث تعريف عقيق ﴾ ۞۞۞۞۞۞۞ كتاب مجمع بحر يازجي حكى سهيل بن عباد قال : بكر يوم كور زاجر .', 'في معمعان ناجر .', 'خوف من اصطكاك هواجر .']


In [20]:
# our variables 
# text_sentences >>> sent_tokenize
#cleaning_sentences >>> cleaning  sentence of  sent_tokenize 
# sentences_words >>> word_tokenize for cleaning_sentences  
#sentences_words >>> list of list after appliny word_tokenize on every sentence 
# steming_list >>> apply steming on sentences_words 
# 
#

In [21]:

model_name = 'microsoft/Multilingual-MiniLM-L12-H384'
import  arabert   



**how work **

In [22]:
from arabert.preprocess import ArabertPreprocessor

model_name = "bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name, keep_emojis=False)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100%|██████████| 241M/241M [00:38<00:00, 6.26MiB/s]


[2023-05-04 17:28:36,379 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


'و+ لن نبالغ إذا قل +نا إن هاتف أو كمبيوتر ال+ مكتب في زمن +نا هذا ضروري'

In [23]:
#from transformers.convert_slow_tokenizer import SentencePieceExtractor

def detect (sent , tools):
  lemmer = lemmatizer.Lemmatizer()
  tagger = naftawayh.wordtag.WordTagger()
  
  
  sent=re.sub(r'[^ء-ي ]',' ',sent)
  words=word_tokenize(sent)
  
  out_list=["كليله", "كذالك", "كليلة"]
  out={}
  out["الجمله"]=sent

  # case one كأن
  for i in range(len(words)) :
    
    
    if words[i] in tools['كأن']:
     
      out["الجمله"]=sent
      out['اداه التشبيه']=words[i]
      # كأن 
      if words[i] =="كأن":
        counter=0
        for x in range(i,len(words)):
          if tagger.is_noun(words[x])  and not (tagger.is_stopword(words[x])) :
            counter+=1
            if counter==1:
              out["المشبه"]= words [x]
            if counter ==2:
              out["المشبه به"]=(words[x])
              break

        

      # كأن + الضمائر
      else:
        out["المشبه"]= "الضمير ف كأن"

        for x in range(i,len(words)):
          if tagger.is_noun(words[x])  and not (tagger.is_stopword(words[x])) :
            out["المشبه به"]=(words[x])
            break
     
      return out 
     

        


## case 2 مثل و يشبه و تشبه :
  for i in range(len(words)) :
 
    if words[i] in tools['case2']:
      
      out["الجمله"]=sent
      out['اداه التشبيه']=words[i]
      
      # يشبه تشبه مثل  
      if words[i] in ["تشبه" ,"مثل","يشبه"]:
        for x in range(i-1,-1,-1):
          if tagger.is_noun(words[x]):
            out["المشبه"]= words[x]
            break
        for j in range(i+1,len(words)):
          if tagger.is_noun(words[j])  :
              out["المشبه به"]= (words[j])
              break
        return out  
      
      else:
        for x in range(i-1,-1,-1):
          if tagger.is_noun(words[x])  :
              out["المشبه"]= words[x]
              break
       
        out["المشبه به"]= "الضمير في اداه التشبيه"
     
      return out 
      

  ##case three حرف الكاف
  for i in range (len(words))  :
   if words[i][0]=='ك' and words[i] not in out_list and arabert_prep.preprocess(words[i])!=words[i] and arabert_prep.preprocess(words[i])[:2]=='ك+'  and lemmer.lemmatize(words[i])[0]!='ك':
      tagger = naftawayh.wordtag.WordTagger();     
      for x in range(i-1,-1,-1):

        if tagger.is_noun(words[x])  :

          out["المشبه"]= words[x]
      for j in range(i,len(words)):
        if tagger.is_noun(words[j])  :
            out["المشبه به"]= (words[j])[1:]
            break
      out["الجمله"]=sent
      out ["اداه التشبيه"]="حرف الكاف"
      return out 
  out['اداه التشبيه']="لا يوجد"
  return out

In [24]:

tools1="كانك  كأنكما  كأنكم  كأنها  كأنه  كأنهما  كأنهم  كأنهن  كأني  كأن"
tools2=" يشبهك يشبهكما يشبهكم يشبهكن  يشبهه يشبهها يشبههما يشبههم  يشبه تشبه مثل  يشبههن تشبههن يشبهني مثلك  مثلكما  مثل مثلكم  مثلكن  مثلها مثلهما  مثلهم  مثلهن مثلي تشبهك تشبهكما تشبهكم تشبهكن  تشبهه تشبهها تشبههما تشبههم تشبههن"
tools3="يشبهك يشبهكما يشبهكم يشبهكن  يشبهه يشبهها يشبههما يشبههم يشبههن تشبههن يشبهني"
tools4="تشبهك تشبهكما تشبهكم تشبهكن  تشبهه تشبهها تشبههما تشبههم تشبههن"

tools1=word_tokenize(tools1)
tools2=word_tokenize(tools2)
tools3=word_tokenize(tools3)
tools4=word_tokenize(tools4)
tools4
all_tools={}
all_tools['كأن']=tools1
all_tools['case2']=tools2
all_tools['يشبه']=tools3
all_tools['تشبه']=tools4




In [25]:

detect(sent=" الابن مثلهم  في العطف ",tools=all_tools)

{'الجمله': ' الابن مثلهم  في العطف ',
 'اداه التشبيه': 'مثلهم',
 'المشبه': 'الابن',
 'المشبه به': 'الضمير في اداه التشبيه'}

In [26]:

detect("احمد كتابه جميل",tools=all_tools)

{'الجمله': 'احمد كتابه جميل', 'اداه التشبيه': 'لا يوجد'}

In [27]:
detect("احمد كالأسد جميل",tools=all_tools)

{'الجمله': 'احمد كالأسد جميل',
 'المشبه': 'احمد',
 'المشبه به': 'الأسد',
 'اداه التشبيه': 'حرف الكاف'}

In [28]:
detect(sent="كليله  ",tools=all_tools)

{'الجمله': 'كليله  ', 'اداه التشبيه': 'لا يوجد'}

# **التشبيه test**

In [29]:
list_for_detecting=[]
for i in cleaning_sentences:
  list_for_detecting.append(detect(sent=i,tools=all_tools))
  
  

In [30]:
for i in list_for_detecting:


  if i['اداه التشبيه']!="لا يوجد" :
    print(i)
    print("_____________________________________________________________________________________________________________________")
    print("______________________________________________________________________________________________________________________")
  

{'الجمله': 'إذا امرأة كأنها من حور الجنان ', 'اداه التشبيه': 'كأنها', 'المشبه': 'الضمير ف كأن', 'المشبه به': 'حور'}
_____________________________________________________________________________________________________________________
______________________________________________________________________________________________________________________
{'الجمله': 'كزجاجة فيها مصباح ', 'المشبه به': 'زجاجة', 'اداه التشبيه': 'حرف الكاف'}
_____________________________________________________________________________________________________________________
______________________________________________________________________________________________________________________


# **التكرار**

In [31]:
def rep(sents, search_range=4):
  sentences=[]
  dic={}
  lemm_sents=lemmatizing(sents)
  for i in range (len(lemm_sents)-1):
    dic={}
    dic['الجمله']=sents[i]
    dic['الكلمات المكرره']=rep_words(sents[i])
    if lemm_sents[i:i+search_range].count(lemm_sents[i])>1:
      dic['هل الجمله مكرره'] ="نعم" 
    else:
      dic['هل الجمله مكرره'] ="لا"  
    sentences.append(dic)
  return sentences

In [32]:
def rep_words(sent):
  li=[]
  words=word_tokenize(sent)
  for i in range(len(words)-1):
    if words.count(words[i])>1 and  words[i] not in li:
      li.append(words[i])
  return li 

# **التكرار test**

In [33]:
duplicated_sentences=[]
sents=sent_tokenize(text)
for i in rep(sents):
  if i['هل الجمله مكرره']=='نعم':
    print(i)
    duplicated_sentences.append(i)
    print("______________________________")

In [34]:
def get_duplicated_sentences(text, search_range=4):
  duplicated_sentences=[]
  sents=sent_tokenize(text)
  lemmat_list=lemmatizing(sents)
  for i in rep(sents,search_range):
    if i['هل الجمله مكرره']=='نعم':
      duplicated_sentences.append(i)
  return duplicated_sentences 

    


# **الجناس**

In [35]:
def detect_Alliteration(word1,word2,r=80):
  ratio=fuzz.ratio(word1,word2)
  if ratio > r and word1 != word2:
    return True 
  else :
    return False

In [36]:
def get_anaphora(sent,r=80):
  count=0
  words=word_tokenize(sent)
  for i in range(len(words)):
    for x in range(i+1,len(words)):
      if (detect_Alliteration(words[i],words[x],r)):
        print(words[i],"      ",words[x])


In [37]:
def detect_anaphora(sent,r=80):
  count=0
  words=word_tokenize(sent)
  for i in range(len(words)):
    for x in range(i+1,len(words)):
      if (detect_Alliteration(words[i],words[x],r)):
        count=1
        return True
  return False 

In [38]:
for i in text_sentences:
  print(i)
  get_anaphora(i,60)
  print("_______________________")


﴿ (3) المقامة الثالثة وتعريف بالعقيقة  ﴾ 
۞۞۞۞۞۞۞
كتاب مجمع البحرين لليازجي
 
 حكى سهيل بن عباد قال: بكرت يوماً بكور الزاجر.
بكرت        بكور
_______________________
في معمعان ناجر.
_______________________
خوفاً من اصطكاك الهواجر.
_______________________
فأمعنت في السياحة.
_______________________
وجعلت أقطع ساحة بعد ساحة.
_______________________
حتى إذا تخللت بعض الغيطان.
_______________________
وقد سال عليها مخاط الشيطان.
_______________________
رأيت كتيبة من الرجال.
_______________________
على كثيب من الرمال.
_______________________
فبذلت في شاكلة الجواد المهماز.
_______________________
ورددت صدور الأرض على الأعجاز.
_______________________
حتى أدركت القوم.
_______________________
في منتصف اليوم.
_______________________
وإذا جنازة قد أودعوها التراب.
_______________________
وشيخ على دكة قد افتتح الخطاب.
_______________________
فقال: يا كرام المعاشر والعشائر.
المعاشر        والعشائر
_______________________
يا كرام المعاشر والعشائر.
المعاشر        والعشائر
_______________________
وأولي ا

In [39]:
def get_weight(text_sentences,r=85):
  num1=0 # التشبيه
  num2=0 # الجناس 
  num3=0 #  التكرار كلمات
  num4=0 # تكرار جمل
  
  for i in text_sentences :
    if detect(i,all_tools)['اداه التشبيه']!="لا يوجد":
      num1=num1+1
    if detect_anaphora(i,r):
      num2=num2+1
    if len(rep_words(i))>0:
      num3=num3+1

  for i in rep(text_sentences):
    if i['هل الجمله مكرره']=='نعم':
      num4=num4+1

  sum=num1+num2+num3+num4
  if sum ==0:
    return 0,0,0,0

  w1=num1/sum
  w2=num2/sum
  w3=num3/sum 
  w4=num4/sum

  return w1,w2,w3,w4

In [40]:
all_sentences =sent_tokenize(text)
w=get_weight(text_sentences,85)

Exception ignored in: <function WordFreqDictionary.__del__ at 0x7fd04090ea70>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/arramooz/wordfreqdictionaryclass.py", line 130, in __del__
    self.db_connect.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140533649946432 and this is thread id 140532932142848.
Exception ignored in: <function ArabicDictionary.__del__ at 0x7fd03e8d9bd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/arramooz/arabicdictionary.py", line 108, in __del__
    self.db_connect.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140533649946432 and this is thread id 140532932142848.
Exception ignored in: <function ArabicDictionary.__del__ at 0x7fd03e8d9bd0>
Traceback (most recent call last):
  File "/usr/local/lib

In [41]:
w

(0.13333333333333333,
 0.26666666666666666,
 0.5333333333333333,
 0.06666666666666667)

In [42]:
def get_score_ARFS(all_sentences , duplicated_sentences,w1=1,w2=1, w3=1 ,w4=1,auto=True,r=85 ):
  dic={}
  socre_of_ARFS=[]
  
  if auto:
    w1,w2,w3,w4=get_weight(all_sentences,85)
  for i in text_sentences :
    score=0
    if detect(i,all_tools)['اداه التشبيه']!="لا يوجد":
      score+=w1
    if detect_anaphora(i,r):
      score+=w2
    if len(rep_words(i))>0:
      score+=w3
    if i in duplicated_sentences:
      score+=w4
    dic['sent']=i
    dic['score']=score
    socre_of_ARFS.append(dic)
    dic={}
  return socre_of_ARFS



  
  
  





In [43]:
socre_of_ARFS=get_score_ARFS (text_sentences, get_duplicated_sentences(text) ,auto=True)

In [44]:
for i in socre_of_ARFS:
  print(i)
  print("_________________________")

{'sent': '﴿ (3) المقامة الثالثة وتعريف بالعقيقة  ﴾ \n۞۞۞۞۞۞۞\nكتاب مجمع البحرين لليازجي\n \n حكى سهيل بن عباد قال: بكرت يوماً بكور الزاجر.', 'score': 0}
_________________________
{'sent': 'في معمعان ناجر.', 'score': 0}
_________________________
{'sent': 'خوفاً من اصطكاك الهواجر.', 'score': 0}
_________________________
{'sent': 'فأمعنت في السياحة.', 'score': 0}
_________________________
{'sent': 'وجعلت أقطع ساحة بعد ساحة.', 'score': 0.5333333333333333}
_________________________
{'sent': 'حتى إذا تخللت بعض الغيطان.', 'score': 0}
_________________________
{'sent': 'وقد سال عليها مخاط الشيطان.', 'score': 0}
_________________________
{'sent': 'رأيت كتيبة من الرجال.', 'score': 0}
_________________________
{'sent': 'على كثيب من الرمال.', 'score': 0}
_________________________
{'sent': 'فبذلت في شاكلة الجواد المهماز.', 'score': 0}
_________________________
{'sent': 'ورددت صدور الأرض على الأعجاز.', 'score': 0}
_________________________
{'sent': 'حتى أدركت القوم.', 'score': 0}
___________________

In [45]:
sum_score_ARFS=0
for  i in socre_of_ARFS :

  sum_score_ARFS+=i['score']
average_score_ARFS=sum_score_ARFS/(len(socre_of_ARFS)+1)
#filter sentences according ARFS score 
for i in socre_of_ARFS:
  if i['score']>(2*average_score_ARFS):
    print(i['sent'])



وجعلت أقطع ساحة بعد ساحة.
واتركوا ما رأيتم نسياً منسياً.
وإلا فالبدار البدار.
وشمروا الذيل وبادروا الوحى 
 من قبل أن يدعوكم داعي الردى 
 واطرحوا كل نعيم وغنى 
 واستهدفوا لوقع أسهم البلى 
 وأقرضوا الله فنعم من وفى 
 ما أجهل الناس وأذهل النهى 
 لو أن هذا المال في هذا الورى 
 قال: ألست ربكم؟ قالوا: بلى 
 ولما فرغ من أبياته زفر زفرة الضرام.
وقالوا له: الدعاء الدعاء!
إذا امرأة كأنها من حور الجنان.
ثم أقسم على القوم فعادوا وكأن على رؤوسهم الطير.
فقلت: إن الشيخ قد أتى الله بقلب سليم: والله يهدي من يشاء إلى صراط مستقيم.
كزجاجة فيها مصباح.
مال على أحد جانبيه وأنشأ يقول: 
 سقى الغمام ترب ذاك القبر 
 فقد سقاني من لذيذ الخمر 
 ما لم أذق نظيره في العمر 
 أفادني في اليوم قبل العصر 
 ما لست أستفيده في الشهر 
 وإن أكن ركبت إثم السكر 
 فقد أفدت القوم عند الذكر 
 مواعظاً تلين صلد الصخر 
 فنلت من ذاك عظيم الأجر 
 وصرت أرجو أن يقوم عذري 
 عند الإله في مقام الحشر 
 بأنني كفرت قبل الوزر 
 قال: فلما فرغ من إنشاده المريب.
طلعت عليه طلعت الذيب.
قلت: فمن التي تشرب الكأس من يديها؟ أحليلة بنيت بها أم خليلة أنست إ

# **get 10% of sentences** 

In [46]:
re=0.10

score=[]
for i in socre_of_ARFS :
  score.append(i['score'])
li_index=[]

for i in range(int(len(socre_of_ARFS)*re)):

  max_index=score.index(max(score))
  li_index.append(max_index)
  score[max_index]=-1
for i in li_index:
  print(socre_of_ARFS[i]['sent'])


  

  






وشمروا الذيل وبادروا الوحى 
 من قبل أن يدعوكم داعي الردى 
 واطرحوا كل نعيم وغنى 
 واستهدفوا لوقع أسهم البلى 
 وأقرضوا الله فنعم من وفى 
 ما أجهل الناس وأذهل النهى 
 لو أن هذا المال في هذا الورى 
 قال: ألست ربكم؟ قالوا: بلى 
 ولما فرغ من أبياته زفر زفرة الضرام.
فقلت: إن الشيخ قد أتى الله بقلب سليم: والله يهدي من يشاء إلى صراط مستقيم.
وجعلت أقطع ساحة بعد ساحة.
وقالوا له: الدعاء الدعاء!
ثم أقسم على القوم فعادوا وكأن على رؤوسهم الطير.
مال على أحد جانبيه وأنشأ يقول: 
 سقى الغمام ترب ذاك القبر 
 فقد سقاني من لذيذ الخمر 
 ما لم أذق نظيره في العمر 
 أفادني في اليوم قبل العصر 
 ما لست أستفيده في الشهر 
 وإن أكن ركبت إثم السكر 
 فقد أفدت القوم عند الذكر 
 مواعظاً تلين صلد الصخر 
 فنلت من ذاك عظيم الأجر 
 وصرت أرجو أن يقوم عذري 
 عند الإله في مقام الحشر 
 بأنني كفرت قبل الوزر 
 قال: فلما فرغ من إنشاده المريب.
طلعت عليه طلعت الذيب.
قلت: فمن التي تشرب الكأس من يديها؟ أحليلة بنيت بها أم خليلة أنست إليها؟ قال: إن بينهما نقطة فلا تحاسب عليها.
واتركوا ما رأيتم نسياً منسياً.
وإلا فالبدار البدار.
إذا امر